# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [106]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from joblib import dump
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head(3)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
df['calls'] = df['calls'].astype(int)

In [7]:
df['messages'] = df['messages'].astype(int)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


Открываем, изучаем датасет, смотрим общее число строк данных, чтобы примерно оценить пропорции разделения на выборки. Приводим значения столбцов calls and mssages к целочисленному типу(уже на автомате делается)

## Разбейте данные на выборки

Тестовая выборка не спрятана, значит будем делить в соотнношении 3:1:1, т.е. 60% данных отдаем обучающей выборке, 20% валидационной, 20% тестовой

In [9]:
features = df.drop(['is_ultra'], axis =1)
target = df['is_ultra']

Обучающая и тестовая выборки

In [10]:
features_train, features_test, target_train, target_test = train_test_split(features,
                                                                              target, 
                                                                              test_size=0.4, 
                                                                              random_state = 12345)

Валидационная выборка

In [11]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test,
                                                                          target_test,
                                                                          test_size = 0.5,
                                                                          random_state = 12345)

In [12]:
print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


In [13]:
print(features_valid.shape)
print(target_valid.shape)

(643, 4)
(643,)


In [14]:
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


## Исследуйте модели

Выполним исследование модели решающего дерева. Создадим цикл для проверки с глубиной от 1 до 5

In [15]:
best_model_t = None
best_result_t = 0
best_depth_t = 0
for depth in range(1, 6):
    model_t = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    model_t.fit(features_train, target_train)
    predictions_valid_t = model_t.predict(features_valid)
    result_t = accuracy_score(target_valid, predictions_valid_t)
    if result_t > best_result_t:
        best_model_t = model_t
        best_depth_t  = depth
        best_result_t = result_t
    
print('best_depth = ', best_depth_t, ':', 'Accuracy лчшей модели ', best_result_t)
    
    

best_depth =  3 : Accuracy лчшей модели  0.7853810264385692


Для решающего дерева лучшая модель с глубиной ветвей 3 и точностью правильных ответов 0,7853

Случайный Лес 

In [16]:
best_model_f = None
best_result_f = 0
best_est_f = 0
best_depth_f = 0
for est in range(1, 15):
    for depth in range(1,11):
        
        model_f = RandomForestClassifier(random_state = 12345, n_estimators = est, max_depth = depth, criterion='gini')
        model_f.fit(features_train, target_train)
        predictions_valid_f = model_f.predict(features_valid)
        result_f = accuracy_score(target_valid, predictions_valid_f)
        if best_result_f < result_f:
            best_model_f = model_f
            best_est_f = est
            best_result_f = result_f
            best_depth_f = depth
        
print('best_est :', best_est_f, ':'
      ,'best_depth :', best_depth_f,
      'Accuracy лучшей модели :', best_result_f)        

best_est : 12 : best_depth : 6 Accuracy лучшей модели : 0.8040435458786936


Для модели случайный лес лучшая точность правильных ответов 0.8040435458786936, с количеством деревьев 12 и глубиной 6

Логистическая регрессия

In [17]:
model_l = LogisticRegression(random_state = 12345, solver='lbfgs', max_iter = 1000)
model_l.fit(features_train, target_train)
predictions_valid_l = model_l.predict(features_valid)

print(accuracy_score(target_valid, predictions_valid_l))

0.7107309486780715


Accuracy логистической регрессии равна 0.7107309486780715, что ниже, чем у дерева решений и слуайного леса.

Вывод:
    Исследование на валидационной выборке показало, что наилучший результат по проверке на Accuracy дает модель случайного леса со следующими параметрами:
    максимальная глубина 6
    количество деревьев 12
    

## Проверьте модель на тестовой выборке

Т.к. модель Случайного леса показала лучший результат ее и будем поверять на тестовой выборке.

In [18]:
model_test = RandomForestClassifier(random_state = 12345, n_estimators = 12, max_depth = 6)
model_test.fit(features_train, target_train)
predictions_test = model_test.predict(features_test)

print(accuracy_score(target_test, predictions_test))

0.7947122861586314


Accuracy модели случайный лес на тестовой выборке получилось 0.7947122861586314 - близко к правде

## (бонус) Проверьте модели на адекватность

In [19]:
model_1 = DummyClassifier(strategy = 'most_frequent')
model_1.fit(features, target)
predictions_1 = model_1.predict(features)

print(accuracy_score(target, predictions_1))

0.693528313627878


Сравниваемая модель показала результат на Accuracy 0.693528313627878, и этого следует, что все исследуемые модели адекватны.

Общий вывод, мы изчили датасет с данными по использованию тарифом мобильного оператора. Затем разбили данные на  выборки, тренировочную, валидационную, на которой ищем наилучшие результаты и тестовую, для проверки вышеперечисленных выборок. Самой удачной моделью с наилучшим показателем оказалась модель случайного леса RF, показатель Accuracy составил 0,8, а у моделей DT и LR - 0.78 и 0,71 соответственно. Тестовая выборка показала наилучший результат 0,79, что очень близко к модели RF. Модель RF проверили на адекватность, сравнили с рандомной моделью, в итоге получили что показатель Accuracy 0.69 ниже, чем у RF (0.8), отсюда сделали вывод, что наша модель прошла проверку на адекваность.  Для рекомендации новых тарифом автоматизированно выгоднее использовать модель Случайного леса.